In [1]:
from coin_game_env import CoinGameEnv
from input_agent import InputAgent
from random_agent import RandomAgent
from play import play_vs_other_agent
from UONI import UONI

In [2]:
env = CoinGameEnv(grid_size=5)
input_agent = InputAgent()

Input Agents

In [ ]:
play_vs_other_agent(env, agent1=input_agent, agent2=input_agent, render=True)

Random Agents

In [9]:
play_vs_other_agent(env, agent1=RandomAgent(1), agent2=RandomAgent(2), render=True)

+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|     | 0   | 1   | 2   | 3   | 4   | 5   | 6   | 7   | 8   |
+=====+=====+=====+=====+=====+=====+=====+=====+=====+=====+
|   0 |     |     |     |     | O   |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|   1 |     |     |     | O   | O   | O   |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|   2 |     |     | O   | O   | O   | O   | O   |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|   3 |     | O   | O   | O   | O   | O   | O   | O   |     |
+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|   4 | O   | O   | O   | O   | O   | O   | O   | O   | O   |
+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|     | 0   | 1   | 2   | 3   | 4   | 5   | 6   | 7   | 8   |
+=====+=====+=====+=====+=====+=====+=====+=====+=====+=====+
|   0 | 

1

Random Agent vs UONI (Universal Optimization Neural Interface)

In [2]:
import matplotlib.pyplot as plt
import wandb
# Assuming ws is a list of winners
def print_graph(ws, level):
    winners = ["UONI", "OTHER"]
    win_counts = [ws.count(x) for x in [1,2]]

    wandb.log({level + " MiniMax wins": win_counts[1],'total': len(ws)})

    plt.bar(winners, win_counts)
    plt.xlabel('Winner Number ' + level)
    plt.ylabel('Win Amount')
    plt.title('Win Amount per Winner')
    plt.show()

    
    

In [ ]:
from UONI import UONI
from random_agent import RandomAgent
from tqdm import tqdm
ws = {'easy':[], 'medium':[], 'hard':[]}
for level in ['easy', 'medium', 'hard']:
    for i in tqdm(range(20)):
        if i%2 == 0:
            ws[level].append(play_vs_other_agent(env, agent1=UONI(1, level), agent2=RandomAgent(2), render=False))
        else:
            ws[level].append(3-play_vs_other_agent(env, agent1=RandomAgent(1), agent2=UONI(2, level), render=False))
    print_graph(ws[level], level)

In [ ]:

from coin_game_env import CoinGameEnv
from MiniMaxAgent import MiniMaxAgent
from play import play_vs_other_agent
from UONI import UONI

env = CoinGameEnv(grid_size=5)

from tqdm import tqdm
ws = {'easy':[], 'medium':[], 'hard':[]}
for level in ['easy', 'medium', 'hard']:
    for i in tqdm(range(500)):
        if i % 2 == 0:
            result = play_vs_other_agent(env, agent1=UONI(1, level), agent2=MiniMaxAgent(3), render=False)
            ws[level].append(result)
        else:
            result = 3 - play_vs_other_agent(env, agent1=MiniMaxAgent(3), agent2=UONI(2, level), render=False)
            ws[level].append(result)
    print_graph(ws[level], level)




In [ ]:
from coin_game_env import CoinGameEnv
from MiniMaxAgent import MiniMaxAgent
from play import play_vs_other_agent
import wandb
from UONI import UONI
from tqdm import tqdm

def train():
    wandb.init()
    config = wandb.config
    env = CoinGameEnv(grid_size=config.grid_size)
    ws = {'easy':[], 'medium':[], 'hard':[]}
    for level in ['easy', 'medium', 'hard']:
        for i in tqdm(range(config.num_episodes)):
            if i % 2 == 0:
                result = play_vs_other_agent(env, agent1=UONI(1, level), agent2=MiniMaxAgent(3), render=False)
                ws[level].append(result)
            else:
                result = 3 - play_vs_other_agent(env, agent1=MiniMaxAgent(3), agent2=UONI(2, level), render=False)
                ws[level].append(result)
        print_graph(ws[level], level)


sweep_config = {
    'method': 'grid',
    'parameters': {
        'grid_size': {
            'values': [3, 4, 6]
        },
        'EvalFunction': {
            'value' : 'PosibleMoves'
        },
        'num_episodes': {
            'values': [200, 500, 1000]
        },
        'depth': {
            'values': [2, 3]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project='coin_game')

wandb.agent(sweep_id, function=train)
